In [7]:
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
import cfscrape
import urllib
from IPython.display import display, display_pretty, Javascript, HTML

# Flow

- [Download and preprocess county-level results](#Townhall-data)
- [Downlaod and preprocess county-level metadata](#Census-data)
- [Combine datasets](#Combine-data)
- [Export county-level results](#Export-data)
- [Visualize](#Visualize)

## Townhall data

In [27]:
# list of state abbreviations
states = ['AL','AK','AZ','AR','CA','CO','CT','DC','DE','FL','GA','HI','ID','IL','IN','IA','KS','KY','LA','ME','MD','MA','MI','MN','MS','MO','MT','NE','NV','NH','NJ','NM','NY','NC','ND','OH','OK','OR','PA','RI','SC','SD','TN','TX','UT','VT','VA','WA','WV','WI','WY']

# list of years
years = ['2010', '2012', '2014', '2016']

# headers for csv export
data = [['state_abbr', 'year', 'candidate', 'party','votes_total']]

In [28]:
# loop through each state's web page http://townhall.com/election/2016/president/%s/county, where %s is the state abbr
for year in years:    
    for state in states:

        scraper = cfscrape.create_scraper()
        r = scraper.get('https://townhall.com/election/' + year + '/senate/' + state + '/county').content
        soup = BeautifulSoup(r, "lxml", from_encoding="utf-8")
        #print soup.prettify()

        # loop through each <table> tag with .ec-table class
        tables = soup.find_all("table", attrs={'class':"table ec-table ec-table-summary"})

        for table in tables:
            if table.findParent("table") is None:
                table_body = table.find('tbody')
                rows = table_body.find_all('tr')
                for row in rows:
                    cols = row.find_all('td')
                    candidate = cols[0].text.strip()
                    party = cols[1]['class'][0]
                    try:
                        total_votes = int(cols[1].text.strip().replace(',','').replace('-','0'))
                    except:
                        total_votes = None
                        

                    #combine each row's results
                    rowData = [state,year,candidate,party, total_votes]
                    data.append(rowData)
        print state
        print year

AL
2010
AK
2010
AZ
2010
AR
2010
CA
2010
CO
2010
CT
2010
DC
2010
DE
2010
FL
2010
GA
2010
HI
2010
ID
2010
IL
2010
IN
2010
IA
2010
KS
2010
KY
2010
LA
2010
ME
2010
MD
2010
MA
2010
MI
2010
MN
2010
MS
2010
MO
2010
MT
2010
NE
2010
NV
2010
NH
2010
NJ
2010
NM
2010
NY
2010
NC
2010
ND
2010
OH
2010
OK
2010
OR
2010
PA
2010
RI
2010
SC
2010
SD
2010
TN
2010
TX
2010
UT
2010
VT
2010
VA
2010
WA
2010
WV
2010
WI
2010
WY
2010
AL
2012
AK
2012
AZ
2012
AR
2012
CA
2012
CO
2012
CT
2012
DC
2012
DE
2012
FL
2012
GA
2012
HI
2012
ID
2012
IL
2012
IN
2012
IA
2012
KS
2012
KY
2012
LA
2012
ME
2012
MD
2012
MA
2012
MI
2012
MN
2012
MS
2012
MO
2012
MT
2012
NE
2012
NV
2012
NH
2012
NJ
2012
NM
2012
NY
2012
NC
2012
ND
2012
OH
2012
OK
2012
OR
2012
PA
2012
RI
2012
SC
2012
SD
2012
TN
2012
TX
2012
UT
2012
VT
2012
VA
2012
WA
2012
WV
2012
WI
2012
WY
2012
AL
2014
AK
2014
AZ
2014
AR
2014
CA
2014
CO
2014
CT
2014
DC
2014
DE
2014
FL
2014
GA
2014
HI
2014
ID
2014
IL
2014
IN
2014
IA
2014
KS
2014
KY
2014
LA
2014
ME
2014
MD
2014
MA
2014
MI
2014


In [29]:
townhall = pd.DataFrame(data) # throw results in dataframe
new_header = townhall.iloc[0] #grab the first row for the header
townhall = townhall[1:] #take the data less the header row
townhall.columns = new_header #set the header row as the df header
townhall['votes_total'] = townhall['votes_total'].astype('float64')
print(townhall.shape[0])
townhall.head()

630


,state_abbr,year,candidate,party,votes_total
1,AL,2010,Richard Shelby,REP,968181.0
2,AL,2010,William Barnes,DEM,515619.0
3,AK,2010,Lisa Murkowski,REP,101091.0
4,AK,2010,Joe Miller,REP,90860.0
5,AK,2010,Scott McAdams,DEM,60045.0


In [30]:
# view by state
townhall[(townhall['state_abbr'] == 'AK')]

,state_abbr,year,candidate,party,votes_total
3,AK,2010,Lisa Murkowski,REP,101091.0
4,AK,2010,Joe Miller,REP,90860.0
5,AK,2010,Scott McAdams,DEM,60045.0
6,AK,2010,Haase,LIB,1459.0
7,AK,2010,Tim Carter,NAF,927.0
8,AK,2010,Ted Gianoutsos,NAF,458.0
9,AK,2010,Sid Hill,W,13.0
340,AK,2014,Dan Sullivan,GOP,119579.0
341,AK,2014,Mark Begich,DEM,111668.0
342,AK,2014,Mark Fish,LIB,9026.0


In [32]:
#print townhall
townhall.to_csv('senate_results_by_state.csv', encoding='utf-8')